# Script writing for 5311 data

In [1]:
import pandas as pd
from siuba import *

from calitp_data_analysis.sql import *
import intake

In [2]:
import _data_prep

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [18]:
#df = _data_prep.load_grantprojects()
vehicles = _data_prep.load_vehicle_data()
# gtfs_status = _data_prep.load_catalog_gtfs()

In [8]:
vehicles.sample()

,Agency,City,State,Legacy NTD ID,NTD ID,Organization Type,Reporter Type,Primary UZA Population,Agency VOMS,Vehicle Type,...,12,13-15,16-20,21-25,26-30,31-60,60+,Total Vehicles,Average Age Of Fleet (In Years),Average Lifetime Miles Per Vehicle
747,Long Beach Transit,Long Beach,CA,9023,90023,Other Publicly-Owned or Privately Chartered Co...,Full Reporter,12150996,196,Bus,...,0,51,13,0,0,0,0,211,8.933649,314947.1185


## Read in Catalog

In [9]:
catalog = intake.open_catalog("catalog.yml")

In [10]:
catalog

? ''
: args:
    path: catalog.yml
  description: ''
  driver: intake.catalog.local.YAMLFileCatalog
  metadata:
    version: 1


In [11]:
gtfs_status = catalog.gtfs_status.read()

In [12]:
gtfs_status = to_snakecase(gtfs_status)

In [13]:
gtfs_status.head()

,name,itp_id,reporting_category,caltrans_district,gtfs_static_status,gtfs_realtime_status,missing_static,services_needing_alerts,services_needing_tripupdates_or_vehiclepositions
0,Long Beach Transit,170.0,Core,07 - Los Angeles,Static Incomplete,RT Incomplete,AquaLink,"Galaxy Express, AquaLink, UCLA/Westwood Commut...","Galaxy Express, AquaLink, UCLA/Westwood Commut..."
1,City of Maywood,196.0,Core,07 - Los Angeles,Static Incomplete,RT Incomplete,Maywood Express Shuttle,"Maywood Dial-A-Ride, Maywood Express Shuttle","Maywood Dial-A-Ride, Maywood Express Shuttle"
2,City of Cerritos,63.0,Core,07 - Los Angeles,Static Incomplete,RT Incomplete,Cerritos on Wheels,"Cerritos on Wheels, Cerritos Dial-A-Ride","Cerritos on Wheels, Cerritos Dial-A-Ride"
3,City of La Puente,152.0,Other Public Transit,07 - Los Angeles,Static Incomplete,RT Incomplete,La Puente Link,"La Puente Link, La Puente Dial-A-Ride","La Puente Link, La Puente Dial-A-Ride"
4,City of Artesia,20.0,Core,07 - Los Angeles,Static Incomplete,RT Incomplete,Artesia Transit,"Artesia Express Dial-A-Ride, Artesia Transit","Artesia Express Dial-A-Ride, Artesia Transit"


In [14]:
gtfs_status.name.nunique()

190

## Read in Vehicle Data

In [15]:
def load_vehiclesdata():
    vehicles_info =  pd.read_excel('gs://calitp-analytics-data/data-analyses/5311 /2020-Vehicles_1.xlsm',
                                   sheet_name = ['Age Distribution'])
    pd.set_option('display.max_columns', None)
    #cannot use to_snakecase because of integer column names
    vehicles_info = vehicles_info['Age Distribution']
    vehicles = (vehicles_info>>filter(_.State=='CA'))
    
    return vehicles

In [16]:
vehicles = load_vehiclesdata()

In [17]:
vehicles.sample(5)

,Agency,City,State,Legacy NTD ID,NTD ID,Organization Type,Reporter Type,Primary UZA Population,Agency VOMS,Vehicle Type,Years Old:,0,1,2,3,4,5,6,7,8,9,10,11,12,13-15,16-20,21-25,26-30,31-60,60+,Total Vehicles,Average Age Of Fleet (In Years),Average Lifetime Miles Per Vehicle
2717,El Dorado County Transit Authority,Diamond Springs,CA,9229,90229,Independent Public Agency or Authority of Tran...,Reduced Reporter,1723634,29,Trucks and other Rubber Tire Vehicles (Service),NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,3,NaN,NaN
4151,City of Calabasas,Calabasas,CA,NaN,90257,"City, County or Local Government Unit or Depar...",Reduced Reporter,12150996,15,Cutaway,NaN,0,0,0,0,0,0,0,0,0,0,0,2,0,2,3,0,0,0,0,7,14.714286,245312.0000
6670,"Wasco, City of",Wasco,CA,9R02-028,9R02-99426,"City, County or Local Government Unit or Depar...",Rural Reporter,0,2,Van,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,8.000000,0.0000
1496,"City of Modesto, dba: Modesto Area Express",Modesto,CA,9007,90007,"City, County or Local Government Unit or Depar...",Full Reporter,358172,69,Minivan,NaN,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,6.000000,99511.0000
858,"City of Fresno, dba: Fresno Area Express",Fresno,CA,9027,90027,"City, County or Local Government Unit or Depar...",Full Reporter,654628,148,Cutaway,NaN,0,0,11,0,0,8,22,0,2,0,0,0,5,3,0,0,0,0,0,51,6.137255,168602.7843


* **note:** multiple entries for each agency

In [19]:
len(vehicles)

705

In [20]:
print(vehicles.Agency.nunique())
#print(vehicle_count.Agency.nunique())

218


## Read in Agency Information

In [21]:
def load_agencyinfo(): 
    agencies = pd.concat(pd.read_excel('gs://calitp-analytics-data/data-analyses/5311 /2020_Agency_Information.xlsx',
                                      sheet_name=None),ignore_index=True)
    agencies = to_snakecase(agencies)
    agency_info = agencies>>filter(_.state=='CA')
    
    return agency_info

In [22]:
agency_info = load_agencyinfo()

In [23]:
agency_info.sample()

,ntd_id,legacy_ntd_id,agency_name,reporter_acronym,doing_business_as,reporter_type,reporting_module,organization_type,reported_by_ntd_id,reported_by_name,subrecipient_type,fy_end_date,original_due_date,address_line_1,address_line_2,p_o__box,city,state,zip_code,zip_code_ext,region,url,fta_recipient_id,duns_number,service_area_sq_miles,service_area_pop,primary_uza,uza_name,tribal_area_name,population,density,sq_miles,voms_do,voms_pt,total_voms,volunteer_drivers,personal_vehicles,tam_tier,number_of_state_counties,number_of_counties_with_service,state_admin_funds_expended
2861,A0003-99446,NaN,Playa Vista Parks and Landscape Corp.,PVPAL,NaN,Reduced Asset Reporter,Asset,Other,A0003,Los Angeles County Metropolitan Transportation...,Asset Subrecipient,2020-06-30,2020-10-31,6200 Playa Vista Dr.,NaN,NaN,Playa Vista,CA,90094.0,NaN,9,playavista.com,5566.0,30611060.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tier II,NaN,NaN,NaN


## Read in Agency Revenue

In [24]:
agency_revenue = pd.concat(pd.read_excel('gs://calitp-analytics-data/data-analyses/5311 /2020_revenue_sources.xlsx',
                                      sheet_name=None), ignore_index=True)

In [25]:
agency_revenue = to_snakecase(agency_revenue)

In [26]:
agency_revenue.sample(5)

,ntd_id,agency_name,reporter_type,subrecipient_type,reporting_module,funding_category,funds_expended_type,total_of_passenger_fares,park_and_ride_revenue__earned_only_,other_agency_revenue__earned_only_,other_agency_revenue_description,auxiliary_revenue___concessions__earned_only_,auxiliary_revenue___advertising__earned_only_,auxiliary_revenue___other__earned_only_,other_auxiliary_revenue_description,non_public_transportation_revenue__earned_only_,non_public_transportation_revenue_description,pt_agmt_with_ntd_reporter_revenue_,pt_agmt_with_non_ntd_reporter_revenue_,subsidy_from_other_sectors_amount,total_recoveries__earned_only_,other_directly_generated_funds__rr_,other_directly_generated_funds_description__rr_,revenue_from_general_fund,state_transportation_funds,income_taxes__earned_only_,sales_taxes__earned_only_,property_taxes__earned_only_,fuel_taxes__earned_only_,other_taxes__earned_only_,other_taxes_description,"bridge,_tunnel,_and_highway_tolls__earned_only_",high_occupancy_tolls__earned_only_,other_dedicated_funds__earned_only_,other_dedicated_funds_description,local_other_funds,local_other_funds_description,rr_local_state_funds,rr_other_funds,rr_other_funds_description,extraordinary_and_special_item_funds,extraordinary_and_special_item_funds_description,fta_capital_program_funds__5309_,arra_fixed_guideway_modernization_funds__5309_,arra_major_capital_investment__new_starts__funds__5309_,american_rescue_plan_capital_investment_grants__5309_,state_of_good_repair__5337_,bus_and_bus_facilities_formula__5339_,fta_urbanized_area_formula__uafp__program__5307_,_5307___capital_assistance_spent_on_operations__including_maintenance_,arra_urbanized_area_program_funds__5307_,arra_urbanized_area_program_funds__5307__capital_assistance_spent_on_operations__including_maintenance_,cares_act_urbanized_area_program_funds__5307_,crrsa_act_urbanized_area_program_funds__5307_,american_rescue_plan_act_of_2021_urbanized_area_program_funds__5307_,fta_enhanced_mobility_of_seniors_and_individuals_with_disabilities_formula_program__5310_,_5310___capital_assistance_spent_on_operations___including_maintenance_expenses_,crrsa_act_enhanced_mobility_of_seniors_and_individuals_with_disabilities_program_funds__5310_,american_rescue_plan_act_of_2021_enhanced_mobility_of_seniors_and_individuals_with_disabilities_program_funds__5310_,fta_formula_grants_for_rural_areas__5311_,fta_formula_grants_for_rural_areas__5311____capital_assistance_spent_on_operations__including_maintenance_,fta_arra_formula_grants_for_rural_areas__5311_,fta_arra_formula_grants_for_rural_areas__5311____capital_assistance_spent_on_operations__including_maintenance_,fta_tribal_transit_funds__5311_,arra_tribal_transit_funds__5311_,cares_act_rural_area_program_funds__5311_,cares_act_public_transportation_on_indian_reservations_program_funds__5311_,crrsa_act_rural_area_program_funds__5311_,crrsa_act_public_transportation_on_indian_reservations_program_funds__5311_,american_rescue_plan_act_of_2021_rural_area_program_funds__5311_,american_rescue_plan_act_of_2021_public_transportation_on_indian_reservations_program_funds__5311_,fta_job_access_and_reverse_commute_formula_program__5316_,fta_new_freedom_program__5317_,_5317___capital_assistance_spent_on_operations___including_maintenance_,fta_metropolitan_planning__5303__,fta_clean_fuels_program__5308_,fta_transit_in_the_park__5320_,other_fta_funds,other_fta_funds_description,other_fta_funds___capital_assistance_spent_on_operations__including_maintenance_,other_fta_funds_cap_asst_spent_on_ops_description,arra_tigger__greenhouse_gas_and_energy_reduction__funds,arra_tiger_multimodal_discretionary_funds,funds_received_from_other_usdot_grant_programs,other_usdot_grants_description,other_federal_funds,other_federal_funds_description,contributed_services,voluntary_non_exchange_transactions,voluntary_non_exchange_transactions_description,sales_and_disposals_of_assets,transportation_development_credits,total
3357,2R02-20983,Wayne Area Transportation Ser

In [27]:
agency_revenue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20446 entries, 0 to 20445
Data columns (total 93 columns):
 #   Column                                                                                                                Non-Null Count  Dtype  
---  ------                                                                                                                --------------  -----  
 0   ntd_id                                                                                                                20446 non-null  object 
 1   agency_name                                                                                                           20446 non-null  object 
 2   reporter_type                                                                                                         20446 non-null  object 
 3   subrecipient_type                                                                                                     9618 non-null   object 
 4   repo

* no state information. have to merge with other dataframes to get the information we need.

## Read BlackCat Data

In [28]:
def load_grantprojects(): 
    df = pd.concat(pd.read_excel('gs://calitp-analytics-data/data-analyses/5311 /Grant_Projects.xlsx', sheet_name=None), ignore_index=True)
    
    pd.set_option('display.max_columns', None)
    
    #a bit of cleaning
    df = to_snakecase(df)
    df = df.drop(columns=['project_closed_by', 'project_closed_date', 'project_closed_time'])
    
    #get just the 5311 agencies
    df = (df>>filter(_.funding_program.str.contains('5311')))
    
    return df

In [29]:
df = load_grantprojects()

In [30]:
df.sample()

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status
96,2016,Section 5311,CA-2017-025 | 0017000158,2016,Sonoma County Transit,BCG0000289,Purchase Replacement < 30 Ft Bus,111204,64BC17-00431,75588.0,75588.0,71949.43,3638.57,0,Open


In [31]:
len(df)

798

In [32]:
df>>count(_.funding_program)

,funding_program,n
0,5311(f) Cont,41
1,5311(f) Round 2,27
2,CMAQ (FTA 5311),31
3,Section 5311,590
4,Section 5311(f),109


In [33]:
df_agency = df>> count(_.organization_name) >> arrange(-_.n)

In [34]:
print(len(df_agency.organization_name.unique()))

88


In [35]:
df_agency.head()

,organization_name,n
60,Mountain Area Regional Transit Authority,33
76,Sonoma County Transit,31
57,Modoc Transportation Agency,28
58,Monterey-Salinas Transit,28
87,Yuba-Sutter Transit Authority,28


In [36]:
sorted(df_agency.organization_name.unique())

['Alpine County Community Development',
 'Amador Transit',
 'Butte County Association of Governments/ Butte Regional Transit',
 'Calaveras County Public Works',
 'Calaveras Transit Agency ',
 'City of Arcata',
 'City of Arvin',
 'City of Auburn',
 'City of California City',
 'City of Chowchilla ',
 'City of Corcoran - Corcoran Area Transit',
 'City of Dinuba ',
 'City of Dixon',
 'City of Escalon ',
 'City of Fairfield',
 'City of Guadalupe',
 'City of McFarland',
 'City of Needles',
 'City of Ojai',
 'City of Porterville',
 'City of Ridgecrest',
 'City of Rio Vista',
 'City of Santa Maria',
 'City of Shafter',
 'City of Solvang',
 'City of Taft',
 'City of Tehachapi',
 'City of Visalia',
 'City of Wasco',
 'City of Woodlake',
 'Colusa County Transit Agency',
 'County Connection (Central Contra Costa Transit Authority)',
 'County of Los Angeles - Department of Public Works',
 'County of Mariposa',
 'County of Nevada Public Works, Transit Services Division',
 'County of Sacramento Depar

* no obvious duplicates or multiple names for one agency

In [37]:
df >> group_by(_.organization_name) >> summarize(n=_.ali.nunique()) >> arrange(-_.n)

,organization_name,n
46,Humboldt Transit Authority,6
18,City of Ojai,5
58,Monterey-Salinas Transit,5
12,City of Dixon,4
54,Madera County,4
...,...,...
80,Town of Truckee,1
81,Transit Joint Powers Authority for Merced County,1
83,Tuolumne County Transit Agency (TCTA),1
84,Victor Valley Transit Authority,1


In [38]:
df >> group_by(_.organization_name) >> summarize(n=_.grant_number.nunique()) >> arrange(-_.n)

,organization_name,n
60,Mountain Area Regional Transit Authority,14
46,Humboldt Transit Authority,13
57,Modoc Transportation Agency,13
39,Eastern Sierra Transit Authority,12
58,Monterey-Salinas Transit,12
...,...,...
50,Klamath Trinity Non-Emergency Transportation​,3
63,Palo Verde Valley Transit Agency,3
14,City of Fairfield,2
53,Livermore Amador Valley Transit Authority,2


## Looking at Agency Names and NTD IDs

In [39]:
agency_crosswalk = agency_info[['agency_name','ntd_id']]

In [40]:
agency_crosswalk>>count(_.agency_name)>>arrange(-_.n)>>filter(_.n>1)

,agency_name,n
146,Humboldt Transit Authority,2
148,Kern Regional Transit,2
165,Los Angeles County Metropolitan Transportation...,2
170,Metropolitan Transportation Commission,2
205,San Luis Obispo Regional Transit Authority,2


In [41]:
agency_crosswalk>>filter(_.agency_name=='Humboldt Transit Authority')

,agency_name,ntd_id
2793,Humboldt Transit Authority,9R02-91036
2870,Humboldt Transit Authority,A0009


In [42]:
#checking to see if the NTD ID is the same in the Vehicles dataset
(vehicles>>filter(_.Agency=='Humboldt Transit Authority'))

,Agency,City,State,Legacy NTD ID,NTD ID,Organization Type,Reporter Type,Primary UZA Population,Agency VOMS,Vehicle Type,Years Old:,0,1,2,3,4,5,6,7,8,9,10,11,12,13-15,16-20,21-25,26-30,31-60,60+,Total Vehicles,Average Age Of Fleet (In Years),Average Lifetime Miles Per Vehicle
4296,Humboldt Transit Authority,Eureka,CA,9R02-016,9R02-91036,Independent Public Agency or Authority of Tran...,Rural Reporter,0,14,Trucks and other Rubber Tire Vehicles (Service),NaN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,NaN,NaN
4297,Humboldt Transit Authority,Eureka,CA,9R02-016,9R02-91036,Independent Public Agency or Authority of Tran...,Rural Reporter,0,14,Cutaway,NaN,0,0,0,2,1,5,0,1,1,0,0,3,0,0,0,0,0,0,0,13,6.384615,0.0
4298,Humboldt Transit Authority,Eureka,CA,9R02-016,9R02-91036,Independent Public Agency or Authority of Tran...,Rural Reporter,0,14,Bus,NaN,0,0,0,1,0,4,5,0,2,2,0,0,0,2,0,0,0,0,0,16,7.062500,0.0
4299,Humboldt Transit Authority,Eureka,CA,9R02-016,9R02-91036,Independent Public Agency or Authority of Tran...,Rural Reporter,0,14,Automobiles (Service),NaN,0,0,0,0,0,0,1,0,1,1,0,0,1,2,1,0,0,0,0,7,NaN,NaN


## Misc.